In [1]:
from kth_dataset import KTH_Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LSTMscratch import LSTMscratch
import torch
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from Model_Wrapper import Wrapper


In [2]:
# Create a dataset and a DataLoader
transforms = transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Normalize((0.5,), (0.5,)),  # Normalize to [-1, 1]
    ])
# Initialize dataset
dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15, transform=transforms,use_saved_samples=True)

dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [3]:
dataset.__len__()


281330

In [4]:
train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=0.2)

In [5]:
datasets = {}

In [6]:
datasets['train'] = Subset(dataset, train_idx)
datasets['val'] = Subset(dataset, val_idx)

In [7]:
train_loader = DataLoader(datasets['train'], batch_size=8, shuffle=True)
test_loader = DataLoader(datasets['train'], batch_size=8, shuffle=False)
print(next(iter(test_loader))[0].size())

torch.Size([8, 15, 1, 64, 64])


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
device = "cpu"

In [11]:
model = LSTMscratch(input_dim=64, hidden_dim=16, number_of_layers=2, device=device)

In [12]:
# model_t = SequentialClassifierWithCells(emb_dim=128, hidden_dim=128, num_layers=2, mode="zeros")
model = model.to(device)


In [13]:
def train_epoch(model, train_loader, optimizer, criterion, epoch, device):
    """ Training a model for one epoch """
    
    loss_list = []
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
        
        progress_bar.set_description(f"Epoch {epoch+1} Iter {i+1}: loss {loss.item():.5f}. ")
        
    mean_loss = np.mean(loss_list)
    return mean_loss, loss_list


@torch.no_grad()
def eval_model(model, eval_loader, criterion, device):
    """ Evaluating the model for either validation or test """
    correct = 0
    total = 0
    loss_list = []
    
    for images, labels in eval_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass only to get logits/output
        outputs = model(images)
                 
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
            
        # Get predictions from the maximum value
        preds = torch.argmax(outputs, dim=1)
        correct += len( torch.where(preds==labels)[0] )
        total += len(labels)
                 
    # Total correct predictions and loss
    accuracy = correct / total * 100
    loss = np.mean(loss_list)
    
    return accuracy, loss

In [14]:
def train_model(model, optimizer, scheduler, criterion, train_loader, valid_loader, num_epochs):
    """ Training a model for a given number of epochs"""
    
    train_loss = []
    val_loss =  []
    loss_iters = []
    valid_acc = []
    
    for epoch in range(num_epochs):
           
        # validation epoch
        model.eval()  # important for dropout and batch norms
        accuracy, loss = eval_model(
                    model=model, eval_loader=valid_loader,
                    criterion=criterion, device=device
            )
        valid_acc.append(accuracy)
        val_loss.append(loss)
        
        # training epoch
        model.train()  # important for dropout and batch norms
        mean_loss, cur_loss_iters = train_epoch(
                model=model, train_loader=train_loader, optimizer=optimizer,
                criterion=criterion, epoch=epoch, device=device
            )
        scheduler.step()
        train_loss.append(mean_loss)
        loss_iters = loss_iters + cur_loss_iters
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"    Train loss: {round(mean_loss, 5)}")
        print(f"    Valid loss: {round(loss, 5)}")
        print(f"    Accuracy: {accuracy}%")
        print("\n")
        save_model(
                model=model,
                optimizer=optimizer,
                epoch=epoch,
                stats={
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                    "loss_iters": loss_iters,
                    "valid_acc": valid_acc,
                }
            )
    
    print(f"Training completed")
    return train_loss, val_loss, loss_iters, valid_acc


def smooth(f, K=5):
    """ Smoothing a function using a low-pass filter (mean) of size K """
    kernel = np.ones(K) / K
    f = np.concatenate([f[:int(K//2)], f, f[int(-K//2):]])  # to account for boundaries
    smooth_f = np.convolve(f, kernel, mode="same")
    smooth_f = smooth_f[K//2: -K//2]  # removing boundary-fixes
    return smooth_f

def count_model_params(model):
    """ Counting the number of learnable parameters in a nn.Module """
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return num_params


def save_model(model, optimizer, epoch, stats):
    """ Saving model checkpoint """
    
    if(not os.path.exists("models")):
        os.makedirs("models")
    savepath = f"models/checkpoint_epoch_{epoch}.pth"

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'stats': stats
    }, savepath)
    return


def load_model(model, optimizer, savepath):
    """ Loading pretrained checkpoint """
    
    checkpoint = torch.load(savepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint["epoch"]
    stats = checkpoint["stats"]
    
    return model, optimizer, epoch, stats

In [15]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [16]:
device

'cpu'

In [19]:
# train_loss, val_loss, loss_iters, valid_acc = train_model(
#         model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion,
#         train_loader=train_loader, valid_loader=test_loader, num_epochs=10
#     )
classes = ["Boxing","HandClapping","HandWaving","Jogging","Running","Walking"]
train = Wrapper(model = model,device = device, criterion = criterion, optimizer = optimizer,writer=None,show_progress_bar= True)
train.train(15,train_loader,test_loader,classes)
train.valid_accuracy()
train.plot_loss_acc()

Epoch 1 Iter 2111: loss 1.45493.:   8%|▊         | 2116/28133 [06:37<1:21:26,  5.32it/s]


KeyboardInterrupt: 